In [10]:
import pandas as pd

import warnings

warnings.filterwarnings("ignore")

# Load Data

In [ ]:
import kagglehub

path = kagglehub.dataset_download("yasserh/customer-segmentation-dataset")

print("Path to dataset files:", path)

100%|██████████| 21.8M/21.8M [01:02<00:00, 363kB/s]

Extracting files...


Path to dataset files: /home/ali/.cache/kagglehub/datasets/yasserh/customer-segmentation-dataset/versions/1


In [9]:
df = pd.read_excel('../data/Online-Retail.xlsx')
df.to_csv('../data/Online-retail.csv', index=False)

In [11]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
